# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Tables](#tables)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb"
    ]
}
```

# **TABLES**

---

### STAGINGS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STAGE SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[DBO].[SEQ_VWSSTAGE_CBS_INHABITANTS_PER_AGEGROUP]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSSTAGE_CBS_INHABITANTS_PER_AGEGROUP]
    START WITH 1
    INCREMENT BY 1;
GO

-- 2) CREATE STAGE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[CBS_INHABITANTS_PER_AGEGROUP]'))
CREATE TABLE [VWSSTAGE].[CBS_INHABITANTS_PER_AGEGROUP](
    [STAGE_ID] INT PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSSTAGE_CBS_INHABITANTS_PER_AGEGROUP]),
    [ID] VARCHAR(100),
	[GESLACHT] VARCHAR(100),
	[LEEFTIJD] VARCHAR(100),
	[BURGERLIJKESTAAT] VARCHAR(100),
	[REGIOS] VARCHAR(100),
	[PERIODEN] VARCHAR(100),
    [BEVOLKINGOP1JANUARI_1] VARCHAR(100),
	[GEMIDDELDEBEVOLKING_2] VARCHAR(100),
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

### STATICS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STATIC SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[DBO].[SEQ_VWSSTATIC_CBS_INHABITANTS_PER_AGEGROUP]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSSTATIC_CBS_INHABITANTS_PER_AGEGROUP]
    START WITH 1
    INCREMENT BY 1;
GO

-- 2) CREATE STATIC TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[CBS_INHABITANTS_PER_AGEGROUP]'))
CREATE TABLE [VWSSTATIC].[CBS_INHABITANTS_PER_AGEGROUP] (
    [ID] INT PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSSTATIC_CBS_INHABITANTS_PER_AGEGROUP]),
    [GESLACHT] VARCHAR(10),
	[LEEFTIJD] VARCHAR(10),
	[BURGERLIJKESTAAT] VARCHAR(10),
	[REGIOS] VARCHAR(10),
	[PERIODEN] VARCHAR(10),
    [BEVOLKINGOP1JANUARI_1] BIGINT,
	[GEMIDDELDEBEVOLKING_2] BIGINT,
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

-- 2.1) ALTER STATIC TABLE(S).....
ALTER TABLE [VWSSTATIC].[CBS_INHABITANTS_PER_AGEGROUP]
    ALTER COLUMN [GEMIDDELDEBEVOLKING_2] FLOAT;
GO

# **STORED PROCEDURES**

---

### STAGE &rarr; STATIC MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT. 
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_INHABITANTS_PER_AGEGROUP]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[CBS_INHABITANTS_PER_AGEGROUP]
    (
		[GESLACHT], 
		[LEEFTIJD], 
		[BURGERLIJKESTAAT], 
		[REGIOS], 
		[PERIODEN], 
		[BEVOLKINGOP1JANUARI_1], 
		[GEMIDDELDEBEVOLKING_2]
	)
    SELECT
        [GESLACHT],
        [LEEFTIJD],
        [BURGERLIJKESTAAT],
        [REGIOS],
        [PERIODEN],
        [BEVOLKINGOP1JANUARI_1],
        CAST([GEMIDDELDEBEVOLKING_2] AS FLOAT)
    FROM [VWSSTAGE].[CBS_INHABITANTS_PER_AGEGROUP]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[CBS_INHABITANTS_PER_AGEGROUP])
END;
GO
